# WebGL feedback

## My isosurface program generates plain old triangles

## -- NOT Triangles which have been translated, rotated, scaled, distorted, and colorized

## To transform plain triangles to fancy triangles we need to push them into the front of the standard pipeline (and maybe use a library like three.js)


<img src="./rasterization1.png">

<a href="https://www.willusher.io/webgl/2019/01/13/volume-rendering-with-webgl">
    https://www.willusher.io/webgl/2019/01/13/volume-rendering-with-webgl</a>
    
## The WebGL2 Transform/Feedback mechanism allows us to "put new processing stages" like Isosurface generation in the front of the standard pipeline.

<hr><hr>

# WebGL2 Feedback:

<h2>WebGL2Feedback allows an HTML5 application to capture outputs from vertex shaders.</h2>

<h2>It can be used to implement special purpose processing pipelines, like isosurface rendering.</h2>

<h2>For example 

# Pipeline Stage 1

In [1]:
import feedback_diagrams
feedback_diagrams.stage1()

DualCanvasWidget(status='deferring flush until render')

## Output from the first stage can be then feed back into the next stage

In [2]:
import feedback_diagrams
feedback_diagrams.stage2()

DualCanvasWidget(status='deferring flush until render')

# This can go on for many stages

# A stage can even get feedbacks from an earlier run of itself (hence the name)

## For particle system simulations or force directed graph relaxation....

<img src="https://media0.giphy.com/media/2bVYAlQiD7TPwDg1Yb/200w.webp?cid=ecf05e47b59a142f56d4c42acb2dfc061e31737fa3965971&rid=200w.webp"/>

<hr>
<hr>
<hr>

# WebGL2 Feedback (and WebGL2 generally) is difficult to program directly

## Low level state machine/device driver interface with a lot of state to manage.

## Lots of setting up pointers and parameters and tearing them down later...

## Here is the "run" method from the feedWebGL wrapper which does a lot of standard things in a standard order:

```Javascript

            run() {
                this.check_input_bindings();
                var program = this.program;
                var gl = program.context.gl;
                gl.useProgram(program.gl_program);
                if (!this.allocated_feedbacks) {
                    this.allocate_feedback_buffers();
                } else {
                    this.bind_feedbacks();
                }
                if (!this.uniforms_installed) {
                    this.install_uniforms();
                }
                this.bind_samplers();
                var mode_name = this.settings.run_type || "POINTS";
                var mode = gl[mode_name];
                var rasterize = this.settings.rasterize;
                if (!rasterize) {
                    gl.enable(gl.RASTERIZER_DISCARD);
                }
                gl.beginTransformFeedback(mode);
                var vertices_per_instance = this.vertices_per_instance;
                var num_instances = this.num_instances;
                if ((num_instances) && (num_instances > 1)) {
                    gl.drawArraysInstanced(mode, 0, vertices_per_instance, num_instances);
                } else {
                    gl.drawArrays(mode, 0, vertices_per_instance);
                }
                gl.endTransformFeedback();
                if (!rasterize) {
                    gl.disable(gl.RASTERIZER_DISCARD);
                }
                gl.bindTransformFeedback(gl.TRANSFORM_FEEDBACK, null);
                this.run_count += 1;  // mainly for debug and test.
            };
```

# *The feedWebGL wrapper API*

## The feedWebGL.js module creates a declarative higher level object hierarchy encapsulating the underlying structures

## Each major concept in the Feedback mechanism is associated with an object implementations.
## Operation sequences are encapsulated as methods of the most appropriate object.
## The API attempts to directly reflect the underlying Feedback paradigm (without higher level abstractions -- it's not another D3.js).

In [3]:
feedback_diagrams.feedback_objects()

DualCanvasWidget(status='deferring flush until render')

# Let's look at a trivial example...